# NLP Analysis of Scientific Abstracts
---
####  US National Library of Medicine National Institutes of Health 
#### The National Center for Biotechnology Information 
#### PubMed

In [1]:
# Import stanard tools
import numpy as np
import pandas as pd
import re

In [2]:
# Import for NLP tools
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Import mallet for topic modeling
mallet_path = '/usr/share/Mallet/bin/mallet'

# Spacy to lemmate
import spacy

# Import NLTK for stop words
from nltk.corpus import stopwords

In [3]:
# Import visualization tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# Import train / test
from sklearn.model_selection import train_test_split

In [5]:
# Import custom module to get abstracts
from lib.get_abstracts import get_abstracts

In [6]:
# Suppress warnings
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [7]:
# Get abstracts and create csv of abstracts
# API key obtained from NCBI
#  get_abstracts('gene therapy', 1000, 'ragorham1@gmail.com', 
#                '3ffbbb6bd110815d69e4aa14b7c26d72ab09')

In [8]:
# Get stop words and add a few more
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [9]:
# Import Dataset
df = pd.read_csv('./data/gene therapy.csv', header=-1, names=['pmid', 
                                                              'abstract'])
df = df.dropna()

In [10]:
df_train, df_test = train_test_split(df)

In [11]:
# Convert to list
data = df_train.abstract.tolist()

In [12]:
# Toeknize sentences to words
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

data_words = list(sent_to_words(data))

In [13]:
# Make bigram and trigram models - threshold is a hyperparameter
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  


bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [14]:
# Remove stop words
data_words_stops = [[word for word in simple_preprocess(str(doc)) \
                       if word not in stop_words] for doc in data_words]

In [15]:
# Form the bigram groups
data_words_bigrams = [bigram_mod[doc] for doc in data_words_stops]

In [16]:
# Form the trigram groups
data_words_trigrams = [trigram_mod[bigram_mod[doc]] for doc in data_words_stops]

In [17]:
# Spacy en model
nlp = spacy.load('en')

In [18]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [19]:
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [20]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [21]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [22]:
doc_lda = lda_model[corpus]

In [25]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, R=10)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      22.592594        1       1  0.093500  0.203142
4      19.847053        1       2  0.192057 -0.110078
7      18.528876        1       3  0.115749  0.118487
3       9.245287        1       4  0.028174 -0.078781
1       8.609173        1       5 -0.140634 -0.043687
5       8.045678        1       6 -0.055750  0.090844
6       7.929904        1       7  0.007703 -0.216309
0       5.201435        1       8 -0.240798  0.036383, topic_info=     Category         Freq            Term        Total  loglift  logprob
term                                                                     
62    Default  1357.000000            cell  1357.000000  10.0000  10.0000
60    Default   597.000000          cancer   597.000000   9.0000   9.0000
148   Default   362.000000           tumor   362.000000   8.0000   8.0000
255   Default   607.000000         patient   607.000000   7.0000   7.0000
3089  Default   155.000000         sup_sup   155.000000   6.0000   6.0000
564   Default   198.000000        mutation   198.000000   5.0000   5.0000
171   Default   380.000000         disease   380.000000   4.0000   4.0000
16    Default   435.000000      expression   435.000000   3.0000   3.0000
911   Default   149.000000           liver   149.000000   2.0000   2.0000
2741  Default   100.000000         sub_sub   100.000000   1.0000   1.0000
1015   Topic1   120.249125          review   121.392220   1.4781  -4.9686
1184   Topic1    66.010725         discuss    66.785634   1.4759  -5.5683
4990   Topic1    57.697058            abcc    58.477096   1.4741  -5.7029
1717   Topic1    54.735474            care    55.510001   1.4735  -5.7556
527    Topic1    48.058190       summarize    48.844811   1.4713  -5.8857
1366   Topic1    41.300500      management    42.075152   1.4690  -6.0373
8675   Topic1    40.103695            dmso    40.878925   1.4684  -6.0667
1181   Topic1    37.655131            cost    38.429542   1.4672  -6.1297
958    Topic1    37.302600       knowledge    38.085512   1.4668  -6.1391
932    Topic1    34.794054           adult    35.569132   1.4655  -6.2087
171    Topic1   323.460745         disease   380.872708   1.3242  -3.9791
982    Topic1    49.774100     alternative    52.317772   1.4377  -5.8507
1307   Topic1    71.970782        disorder    79.127946   1.3927  -5.4819
373    Topic1   200.700743         genetic   265.142079   1.2091  -4.4563
532    Topic1    76.004210            year    86.161725   1.3621  -5.4274
27     Topic1    75.318210            need    91.243469   1.2957  -5.4364
46     Topic1   338.947671         therapy   707.136824   0.7522  -3.9323
5      Topic1   144.673541        clinical   230.897714   1.0201  -4.7837
200    Topic1   136.396183         include   219.884237   1.0100  -4.8426
147    Topic1   202.236373       treatment   407.561895   0.7868  -4.4487
...       ...          ...             ...          ...      ...      ...
3089   Topic7   152.674474         sup_sup   155.976364   2.5131  -3.6829
5016   Topic7    33.385362             ccr    34.166876   2.5114  -5.2031
2729   Topic7    27.441314  mineralization    28.221926   2.5065  -5.3991
1492   Topic7    23.444575    osteosarcoma    24.228230   2.5016  -5.5565
201    Topic7    42.134811    inflammatory    47.225309   2.4205  -4.9703
61     Topic7    56.092223              cd    72.272635   2.2811  -4.6842
117    Topic7    71.613794           mouse   141.739119   1.8518  -4.4399
62     Topic7   151.587226            cell  1357.996543   0.3419  -3.6900
950    Topic7    57.363429        increase   274.761600   0.9680  -4.6618
87     Topic7    41.573930         express    64.550323   2.0946  -4.9837
101    Topic7    45.643931          induce   183.119741   1.1453  -4.8903
6036   Topic8    19.143993  antiretroviral    19.942814   2.9154  -5.3375
710    Topic8    18.136513           index    18.922804   2.9138  -5.3915
8689   Topic8

In [24]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.00545879446082

Coherence Score:  0.3924419752363235
